In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import log_loss
from sklearn.cross_validation import KFold,StratifiedKFold
import pickle
from sklearn.ensemble import ExtraTreesClassifier as ETC


/opt/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#lodaing data
data_path = "/home/raku/kaggleData/2sigma/xgb142/"
store = "/home/raku/kaggleData/2sigma/et2000/"
train_file = data_path + "xgb1.42-train.json"
test_file = data_path + "xgb1.42-test.json"
train_df = pd.read_json(train_file)
test_df = pd.read_json(test_file)
print train_df.shape
print test_df.shape

feature_file = data_path+'xgb142features.pickle'
fileObject = open(feature_file,'r') 
features = pickle.load(fileObject)
fileObject.close()
print len(features)

(49352, 293)
(74659, 292)
220


In [3]:
#running and getting the cv from xgboost
target_num_map = {'high':0, 'medium':1, 'low':2}

train_y = np.array(train_df['interest_level'].apply(lambda x: target_num_map[x]))

KF=StratifiedKFold(train_y,5,shuffle=True,random_state = 2333)

In [ ]:
cv_scores=[]
i=0

for dev_index, val_index in KF: 
    result_dict = {}
    
    dev_set, val_set = train_df.iloc[dev_index,:] , train_df.iloc[val_index,:] 
       #filter the features
    dev_X, val_X = dev_set[features].as_matrix(), val_set[features].as_matrix()
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    
    et = ETC(2000,random_state=0)
    et.fit(dev_X,dev_y)
    preds = et.predict_proba(val_X)

    #save the pickles for futures use
    pickl_file = store+'et2000-5fold-out-'+str(i)+'.pickle'
    fileObject = open(pickl_file,'wb') 
    pickle.dump(preds,fileObject)   
    fileObject.close()

    loss = log_loss(val_y, preds)
    
    cv_scores.append(loss)
    i+=1
    print'loss for the turn '+str(i)+' is '+str(loss)

print 'The mean of the cv_scores is:'
print np.mean(cv_scores)

loss for the turn 1 is 0.59836280628
loss for the turn 2 is 0.60035716028
loss for the turn 3 is 0.606483789898
loss for the turn 4 is 0.606375107013
loss for the turn 5 is 0.593978481942
The mean of the cv_scores is:
0.601111469083


In [ ]:
train_X, test_X = train_df[features].as_matrix(), test_df[features].as_matrix()

et = ETC(2000)
et.fit(train_X,train_y)
preds = et.predict_proba(test_X)


out_df = pd.DataFrame(preds)
out_df.columns = ["high", "medium", "low"]
out_df["listing_id"] = test_df.listing_id.values
out_df.to_json(store+'et2000-bulk-out.json')


In [6]:
test_out = pd.read_json(store+'et2000-bulk-out.json')

In [7]:
test_out

,high,listing_id,low,medium
0,0.0930,7142618,0.4075,0.4995
1,0.0110,7210040,0.9500,0.0390
10,0.0020,6832604,0.9755,0.0225
100,0.0675,6830595,0.7645,0.1680
1000,0.0035,6843709,0.9780,0.0185
10000,0.1880,7232076,0.5205,0.2915
10001,0.2010,6832266,0.3920,0.4070
10002,0.0275,6931714,0.5235,0.4490
10003,0.6090,7316357,0.1565,0.2345
10004,0.0230,6839137,0.8710,0.1060
